In [3]:
import os
import numpy as np
import pandas as pd
import torch
import os
import json
import pandas as pd
from PIL import Image
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
import cv2
import numpy as np
%matplotlib inline
from tqdm import tqdm
import glob
import pickle

In [4]:
DATA_DIR = '/mnt/chicm/data/open-images/relation'
DETECT_DATA_DIR = '/mnt/chicm/data/open-images/detect'

In [38]:
!ls sub_detect_57 -lh

total 1.3G
-rw-rw-r-- 1 core core 266M Jul 22 16:57 sub_detect_0722_1.csv
-rw-rw-r-- 1 core core 252M Jul 24 17:01 sub_detect_0724_lb05524.csv
-rw-rw-r-- 1 core core 233M Jul 30 08:18 sub_detect_57_0730_lb05849.csv
-rw-rw-r-- 1 core core 259M Aug  5 07:41 sub_detect_57_cas_1024_0805_1_lb06012.csv
-rw-rw-r-- 1 core core 240M Aug  4 13:36 sub_detect_cas_1024_57_0804_lb06067.csv


In [39]:
df1 = pd.read_csv('sub_detect_57/sub_detect_cas_1024_57_0804_lb06067.csv')
df2 = pd.read_csv('sub_detect_57/sub_detect_57_0730_lb05849.csv')
df3 = pd.read_csv('sub_detect_57/sub_detect_0724_lb05524.csv')
df1.head()

,ImageId,PredictionString
0,21135d5ed0e7a0b7,/m/04yx4 0.8447121 0.3793392 0.2354858 0.75110...
1,763c72626bf24534,/m/04yx4 0.9533325 0.0052673 0.1912388 0.25071...
2,7540c162180973b1,/m/04yx4 0.6267715 0.7935570 0.1335864 0.85723...
3,c4e522dce52cb02f,/m/04yx4 0.8973290 0.3921414 0.2635090 0.54854...
4,cafcd9496c808085,/m/0k4j 0.0023795 0.7963226 0.0179788 0.997849...


In [40]:
#ImageID,PredictionString
#ImageID,{Label Confidence XMin YMin XMax YMax} {...}

In [41]:
def get_dets_by_pred_str(pred_str):
    try:
        if len(pred_str) < 1:
            return []
    except:
        print('pred_str:', pred_str)
        raise
    det_items = pred_str.split(' ')
    dets, det = [], []
    for i in range(len(det_items)):
        det.append(det_items[i])
        if (i+1) % 6 == 0:
            dets.append(det)
            det = []
    return dets

In [42]:
df1['dets'] = df1.PredictionString.fillna('').map(lambda x: get_dets_by_pred_str(x))
df2['dets'] = df2.PredictionString.fillna('').map(lambda x: get_dets_by_pred_str(x))
df3['dets'] = df3.PredictionString.fillna('').map(lambda x: get_dets_by_pred_str(x))
df1.head()

,ImageId,PredictionString,dets
0,21135d5ed0e7a0b7,/m/04yx4 0.8447121 0.3793392 0.2354858 0.75110...,"[[/m/04yx4, 0.8447121, 0.3793392, 0.2354858, 0..."
1,763c72626bf24534,/m/04yx4 0.9533325 0.0052673 0.1912388 0.25071...,"[[/m/04yx4, 0.9533325, 0.0052673, 0.1912388, 0..."
2,7540c162180973b1,/m/04yx4 0.6267715 0.7935570 0.1335864 0.85723...,"[[/m/04yx4, 0.6267715, 0.7935570, 0.1335864, 0..."
3,c4e522dce52cb02f,/m/04yx4 0.8973290 0.3921414 0.2635090 0.54854...,"[[/m/04yx4, 0.8973290, 0.3921414, 0.2635090, 0..."
4,cafcd9496c808085,/m/0k4j 0.0023795 0.7963226 0.0179788 0.997849...,"[[/m/0k4j, 0.0023795, 0.7963226, 0.0179788, 0...."


In [43]:
len(df1.iloc[0].dets)

50

In [44]:
def dets_to_xywh(dets):
    res = []
    for det in dets:
        label = det[0]
        conf = float(det[1])
        xmin = float(det[2])
        ymin = float(det[3])
        xmax = float(det[4])
        ymax = float(det[5])
        new_det = [(xmin+xmax)/2, (ymin+ymax)/2, xmax-xmin, ymax-ymin, label, conf]
        res.append(new_det)
    return res                                                                 

In [45]:
dets1 = df1.dets.map(lambda x: dets_to_xywh(x))
dets2 = df2.dets.map(lambda x: dets_to_xywh(x))
dets3 = df3.dets.map(lambda x: dets_to_xywh(x))

In [14]:
len(dets3)

99999

In [48]:
final_ens = []
for i in tqdm(range(len(dets1))):
    ens = GeneralEnsemble([dets1[i], dets2[i], dets3[i]], weights=[0.5, 0.3, 0.2])
    final_ens.append(ens)


  1%|▏         | 1411/99999 [00:16<21:57, 74.84it/s]


  3%|▎         | 2940/99999 [00:33<15:48, 102.31it/s]


  4%|▍         | 4363/99999 [00:49<20:05, 79.36it/s]


  6%|▌         | 5886/99999 [01:05<14:40, 106.95it/s]


  7%|▋         | 7346/99999 [01:22<19:27, 79.35it/s]


...

 94%|█████████▍| 94406/99999 [17:40<01:11, 77.93it/s]


 96%|█████████▌| 95917/99999 [17:57<00:32, 123.74it/s]


 97%|█████████▋| 97408/99999 [18:13<00:31, 81.46it/s]


 99%|█████████▉| 98916/99999 [18:29<00:10, 106.48it/s]


100%|██████████| 99999/99999 [18:41<00:00, 89.16it/s] 

In [25]:
final_ens[0]

[[0.56287375,
  0.6156048166666667,
  0.3732778333333333,
  0.7542146999999999,
  '/m/04yx4',
  0.8358440666666666],
 [0.20421474999999994,
  0.5516028833333333,
  0.39796663333333326,
  0.8459136333333332,
  '/m/04yx4',
  0.8155694999999998],
 [0.5612155333333333,
  0.3895833,
  0.3062776666666666,
  0.5972839999999999,
  '/m/04yx4',
  0.38746346666666664],
 [0.7478497333333333,
  0.4802543999999999,
  0.17359593333333334,
  0.6497441333333334,
  '/m/04yx4',
  0.4323228333333333],
 [0.6031168,
  0.31530535000000004,
  0.22100509999999995,
  0.4474377,
  '/m/04yx4',
  0.22904216666666666],
 [0.4723853333333333,
  0.5754557166666665,
  0.7602043333333333,
  0.8450487666666666,
  '/m/04yx4',
  0.2635818333333333],
 [0.6127747833333334,
  0.19058814999999998,
  0.18288163333333332,
  0.19590343333333332,
  '/m/04yx4',
  0.3645910333333333],
 [0.8986425833333332,
  0.5280160333333332,
  0.19877376666666663,
  0.7673663999999999,
  '/m/04yx4',
  0.24447426666666666],
 [0.6502732166666667,
 

In [26]:
top_50_ens = []
for i in tqdm(range(len(final_ens))):
    dets = sorted(final_ens[i], key=lambda x: x[5], reverse=True)[:50]
    top_50_ens.append(dets)


100%|██████████| 99999/99999 [00:02<00:00, 45083.93it/s]

In [27]:
len(top_50_ens[0])

50

In [49]:
def xywh_to_pre_str(dets):
    res = []
    if len(dets) < 1:
        return ''
    for det in dets:
        label = det[4]
        conf = det[5]
        xmin = det[0] - (det[2] / 2)
        xmax = det[0] + (det[2] / 2)
        ymin = det[1] - (det[3] / 2)
        ymax = det[1] + (det[3] / 2)
        
        res.append(label)
        res.append('{:.7f}'.format(conf))
        res.append('{:.7f}'.format(xmin))
        res.append('{:.7f}'.format(ymin))
        res.append('{:.7f}'.format(xmax))
        res.append('{:.7f}'.format(ymax))

    res = [str(x) for x in res]
    return ' '.join(res)

In [50]:
pred_strings = pd.Series(final_ens).map(lambda x: xywh_to_pre_str(x)).values
len(pred_strings)

99999

In [30]:
pred_strings_top50 = pd.Series(top_50_ens).map(lambda x: xywh_to_pre_str(x)).values
len(pred_strings_top50)

99999

In [51]:
df_sub = pd.read_csv(os.path.join(DATA_DIR, 'VRD_sample_submission.csv'))
df_sub.PredictionString = pred_strings
df_sub.head()

,ImageId,PredictionString
0,21135d5ed0e7a0b7,/m/04yx4 0.8314903 0.3767009 0.2384787 0.74942...
1,763c72626bf24534,/m/04yx4 0.9631467 0.0057713 0.1895752 0.24743...
2,7540c162180973b1,/m/04yx4 0.6183816 0.7953903 0.1339338 0.85704...
3,c4e522dce52cb02f,/m/04yx4 0.8953303 0.3924925 0.2646109 0.54842...
4,cafcd9496c808085,/m/0k4j 0.0014457 0.7976004 0.0172362 0.998184...


In [52]:
df_sub.to_csv('ensemble_detect_57_0808_2_with_weights.csv', columns=['ImageId', 'PredictionString'], index=False)

In [53]:
!kaggle competitions submit -c open-images-2019-object-detection -f ensemble_detect_57_0808_2_with_weights.csv -m "weights: 0.5, 0.3, 0.2"

100%|████████████████████████████████████████| 421M/421M [00:17<00:00, 25.4MB/s]
Successfully submitted to Open Images 2019 - Object Detection

In [35]:
df_sub.PredictionString = pred_strings_top50
df_sub.to_csv('ensemble_detect_57_0808_1_top50.csv', columns=['ImageId', 'PredictionString'], index=False)

In [36]:
!kaggle competitions submit -c open-images-2019-object-detection -f ensemble_detect_57_0808_1_top50.csv -m "submit"

100%|████████████████████████████████████████| 267M/267M [00:13<00:00, 20.3MB/s]
Successfully submitted to Open Images 2019 - Object Detection

In [37]:
!kaggle competitions submissions -c open-images-2019-object-detection

fileName                                     date                 description  status    publicScore  privateScore  
-------------------------------------------  -------------------  -----------  --------  -----------  ------------  
ensemble_detect_57_0808_1_top50.csv          2019-08-08 09:56:38  submit       complete  0.06088      None          
ensemble_detect_57_0808_1.csv                2019-08-08 09:54:15  submit       complete  0.06155      None          
sub_detect_57_cas_1024_0805_1.csv            2019-08-05 07:52:30  submit       complete  0.06012      None          
sub_detect_cas_1024_57_0804.csv              2019-08-04 13:37:48  submit       complete  0.06067      None          
sub_detect_0724.csv                          2019-08-03 00:41:03  test         complete  0.05524      None          
sub_detect_57_0730.csv                       2019-08-03 00:39:53  test         complete  0.05849      None          
sub_cas_0_500_0729.csv                       2019-07-28 17:23:42

In [18]:
""" 
Ensembling methods for object detection.
"""

""" 
General Ensemble - find overlapping boxes of the same class and average their positions
while adding their confidences. Can weigh different detectors with different weights.
No real learning here, although the weights and iou_thresh can be optimized.

Input: 
 - dets : List of detections. Each detection is all the output from one detector, and
          should be a list of boxes, where each box should be on the format 
          [box_x, box_y, box_w, box_h, class, confidence] where box_x and box_y 
          are the center coordinates, box_w and box_h are width and height resp.
          The values should be floats, except the class which should be an integer.

 - iou_thresh: Threshold in terms of IOU where two boxes are considered the same, 
               if they also belong to the same class.
               
 - weights: A list of weights, describing how much more some detectors should
            be trusted compared to others. The list should be as long as the
            number of detections. If this is set to None, then all detectors
            will be considered equally reliable. The sum of weights does not
            necessarily have to be 1.

Output:
    A list of boxes, on the same format as the input. Confidences are in range 0-1.
"""
def GeneralEnsemble(dets, iou_thresh = 0.5, weights=None):
    assert(type(iou_thresh) == float)
    
    ndets = len(dets)
    
    if weights is None:
        w = 1/float(ndets)
        weights = [w]*ndets
    else:
        assert(len(weights) == ndets)
        
        s = sum(weights)
        for i in range(0, len(weights)):
            weights[i] /= s

    out = list()
    used = list()
    
    for idet in range(0,ndets):
        det = dets[idet]
        for box in det:
            if box in used:
                continue
                
            used.append(box)
            # Search the other detectors for overlapping box of same class
            found = []
            for iodet in range(0, ndets):
                odet = dets[iodet]
                
                if odet == det:
                    continue
                
                bestbox = None
                bestiou = iou_thresh
                for obox in odet:
                    if not obox in used:
                        # Not already used
                        if box[4] == obox[4]:
                            # Same class
                            iou = computeIOU(box, obox)
                            if iou > bestiou:
                                bestiou = iou
                                bestbox = obox
                                
                if not bestbox is None:
                    w = weights[iodet]
                    found.append((bestbox,w))
                    used.append(bestbox)
                            
            # Now we've gone through all other detectors
            if len(found) == 0:
                new_box = list(box)
                new_box[5] /= ndets
                out.append(new_box)
            else:
                allboxes = [(box, weights[idet])]
                allboxes.extend(found)
                
                xc = 0.0
                yc = 0.0
                bw = 0.0
                bh = 0.0
                conf = 0.0
                
                wsum = 0.0
                for bb in allboxes:
                    w = bb[1]
                    wsum += w

                    b = bb[0]
                    xc += w*b[0]
                    yc += w*b[1]
                    bw += w*b[2]
                    bh += w*b[3]
                    conf += w*b[5]
                
                xc /= wsum
                yc /= wsum
                bw /= wsum
                bh /= wsum    

                new_box = [xc, yc, bw, bh, box[4], conf]
                out.append(new_box)
    return out
    
def getCoords(box):
    x1 = float(box[0]) - float(box[2])/2
    x2 = float(box[0]) + float(box[2])/2
    y1 = float(box[1]) - float(box[3])/2
    y2 = float(box[1]) + float(box[3])/2
    return x1, x2, y1, y2
    
def computeIOU(box1, box2):
    x11, x12, y11, y12 = getCoords(box1)
    x21, x22, y21, y22 = getCoords(box2)
    
    x_left   = max(x11, x21)
    y_top    = max(y11, y21)
    x_right  = min(x12, x22)
    y_bottom = min(y12, y22)

    if x_right < x_left or y_bottom < y_top:
        return 0.0    
        
    intersect_area = (x_right - x_left) * (y_bottom - y_top)
    box1_area = (x12 - x11) * (y12 - y11)
    box2_area = (x22 - x21) * (y22 - y21)        
    
    iou = intersect_area / (box1_area + box2_area - intersect_area + 1e-6)
    return iou
    
if __name__=="__main__":
    # Toy example
    dets = [ 
            [[0.1, 0.1, 1.0, 1.0, 1, 0.9], [1.2, 1.4, 0.5, 1.5, 1, 0.9]],
            [[0.2, 0.1, 0.9, 1.1, 1, 0.8],[1.19, 1.4, 0.5, 1.5, 1, 0.9]],
            [[5.0,5.0,1.0,1.0,1,0.5]]
           ]
    
    ens = GeneralEnsemble(dets, weights = [1.0, 0.1, 0.5])
    print(ens)

[[0.10909090909090909, 0.1, 0.990909090909091, 1.009090909090909, 1, 0.6125], [1.199090909090909, 1.4000000000000001, 0.5, 1.5, 1, 0.61875], [5.0, 5.0, 1.0, 1.0, 1, 0.16666666666666666]]


In [ ]:
[[0.10909090909090909, 0.1, 0.990909090909091, 1.009090909090909, 'a', 0.6125], [1.2, 1.4, 0.5, 1.5, 'a', 0.3], [5.0, 5.0, 1.0, 1.0, 'a', 0.16666666666666666]]